In [67]:
import numpy as np
import numpy.ma as ma
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split


Preparing the data

In [68]:
# importing the data
item_train = pd.read_csv('content_item_train.csv')
user_train = pd.read_csv('content_user_train.csv')
y_train = pd.read_csv('content_y_train.csv')

num_user_features = user_train.shape[1] - 3  # remove userid, rating count and ave rating during training
num_item_features = item_train.shape[1] - 1  # remove movie id at train time
uvs = 3  # user genre vector start
ivs = 3  # item genre vector start
u_s = 3  # start of columns to use in training, user
i_s = 1  # start of columns to use in training, items
print(f"Number of training vectors: {len(item_train)}")



Number of training vectors: 50883


In [69]:
print("num_user_features", num_user_features)

num_user_features 14


In [70]:
# scale training data
item_train_unscaled = item_train
user_train_unscaled = user_train
y_train_unscaled    = y_train

scalarItem = StandardScaler()
scalarItem.fit(item_train)
item_train = scalarItem.transform(item_train)

scalarUser = StandardScaler()
scalarUser.fit(user_train)
user_train = scalarUser.transform(user_train)

# Convert y_train to a NumPy array and reshape it
y_train_array = y_train.values.reshape(-1, 1)


scalarTarget = MinMaxScaler((-1,1))
scalarTarget.fit(y_train_array)
y_train= scalarTarget.transform(y_train_array)


print(np.allclose(item_train_unscaled, scalarItem.inverse_transform(item_train)))
print(np.allclose(user_train_unscaled, scalarUser.inverse_transform(user_train)))

True
True


In [71]:
item_train, item_test = train_test_split(item_train, train_size=0.80, shuffle=True, random_state=1)
user_train, user_test = train_test_split(user_train, train_size=0.80, shuffle=True, random_state=1)
y_train, y_test       = train_test_split(y_train,    train_size=0.80, shuffle=True, random_state=1)
print(f"movie/item training data shape: {item_train.shape}")
print(f"movie/item test data shape: {item_test.shape}")

movie/item training data shape: (40706, 17)
movie/item test data shape: (10177, 17)


In [72]:
print("Shape of user_train slice:", user_train[:, u_s:].shape)
print("Shape of item_train slice:", item_train[:, i_s:].shape)

Shape of user_train slice: (40706, 14)
Shape of item_train slice: (40706, 16)


In [73]:
from tensorflow.keras import layers

num_outputs = 32

tf.random.set_seed(1)

class L2NormalizationLayer(layers.Layer):
    def call(self, inputs):
        return tf.linalg.l2_normalize(inputs, axis=1)

user_NN = tf.keras.Sequential([
    tf.keras.layers.Dense(256,activation='relu'),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(num_outputs),
])

item_NN = tf.keras.Sequential([
    tf.keras.layers.Dense(256,activation='relu'),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(num_outputs),
])

# create the user input and point to the base network

input_user = tf.keras.Input(shape=(num_user_features,))
vu = user_NN(input_user)
vu = L2NormalizationLayer()(vu)

# create the item input and point to the base network

input_item = tf.keras.Input(shape=(num_item_features,))
vm = item_NN(input_item)
vm = L2NormalizationLayer()(vm)

# compute the dot product of the two vectors vu and vm

output = tf.keras.layers.Dot(axes=1)([vu, vm])

# specify the inputs and output of the model

model = tf.keras.Model(inputs=[input_user, input_item], outputs=output)

model.summary()


Model: "functional_22"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_37      │ (None, 14)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_39      │ (None, 16)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_40       │ (None, 32)        │     40,864 │ input_layer_37[0… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_41       │ (None, 32)        │     41,376 │ input_layer_39[0… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ l2_normalization_l… │ (None, 32)        │          0 │ sequential_40[0]… │
│ (L2NormalizationLa… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ l2_normalization_l… │ (None, 32)        │          0 │ sequential_41[0]… │
│ (L2NormalizationLa… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dot_3 (Dot)         │ (None, 1)         │          0 │ l2_normalization… │
│                     │                   │            │ l2_normalization… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 82,240 (321.25 KB)

 Trainable params: 82,240 (321.25 KB)

 Non-trainable params: 0 (0.00 B)

In [75]:
tf.random.set_seed(1)
cost_fn = tf.keras.losses.MeanSquaredError()
opt = keras.optimizers.Adam(learning_rate=0.1)
model.compile(optimizer=opt,
              loss=cost_fn)

In [77]:
tf.random.set_seed(1)
model.fit([user_train[:, u_s:], item_train[:, i_s:]], y_train, epochs=50)

Epoch 1/50
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0614
Epoch 2/50
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0611
Epoch 3/50
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0610
Epoch 4/50
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0609
Epoch 5/50
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.0607
Epoch 6/50
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0606
Epoch 7/50
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 0.0605
Epoch 8/50
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - loss: 0.0603
Epoch 9/50
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - loss: 0.0601
Epoch 10/50
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 0.0600
Epoch 11/50
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 0.0598
Epoch 12/50
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - loss: 0.0597
Epoch 13/50
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - loss: 0.0595
Epoch 14/50
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - loss: 0.0593
Epoch 15/50
1273/1273 ━━━━━━━

In [78]:
model.evaluate([user_test[:, u_s:], item_test[:, i_s:]], y_test)

319/319 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0824


0.07984159886837006

In [79]:
new_user_id = 5000
new_rating_ave = 0.0
new_action = 0.0
new_adventure = 5.0
new_animation = 0.0
new_childrens = 0.0
new_comedy = 0.0
new_crime = 0.0
new_documentary = 0.0
new_drama = 0.0
new_fantasy = 5.0
new_horror = 0.0
new_mystery = 0.0
new_romance = 0.0
new_scifi = 0.0
new_thriller = 0.0
new_rating_count = 3

user_vec = np.array([[new_user_id, new_rating_count, new_rating_ave,
                      new_action, new_adventure, new_animation, new_childrens,
                      new_comedy, new_crime, new_documentary,
                      new_drama, new_fantasy, new_horror, new_mystery,
                      new_romance, new_scifi, new_thriller]])